In [1]:
from pathlib import Path
import os
import pandas as pd
import sys
import requests
from functools import partial
from tqdm import tqdm

import torch
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch 
from einops import rearrange, repeat
from loguru import logger
from torchmetrics import Metric, BootStrapper
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import seaborn as sns
from matplotlib.pyplot import figure
from cka import gram, centering_mat, centered_gram, unbiased_hsic_xy, MinibatchCKA, HookedCache, make_pairwise_metrics, update_metrics, get_simmat_from_metrics
import numpy as np
import re

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import timm
from timm.utils import accuracy
from torch.utils.tensorboard import SummaryWriter

#from get_models import get_model

writer = SummaryWriter()

DEVICE = torch.device('cpu')
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')

## Compare COCO results

In [2]:
from ast import literal_eval

In [3]:
results = {'MAE': [], 'MoCoV3': [], 'DINO': []}

for key in results:
    folder_name = "/checkpoint/sshkhr/experiments/detectron/" + key.lower() + "/output/"
    log_file = folder_name + "metrics.json"

    print(log_file)
    if os.path.isfile(log_file): 
        with open(log_file, "r") as log:
            last_line = log.readlines()[-1]
            log_dict = literal_eval(last_line)
               
    results[key] = log_dict

/checkpoint/sshkhr/experiments/detectron/mae/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/mocov3/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/dino/output/metrics.json


In [4]:
df = pd.DataFrame.from_dict(results['MAE'], orient='index', columns=['MAE'])
df['MoCo'] = pd.Series(results['MoCoV3'])
df['DINO'] = pd.Series(results['DINO'])

In [5]:
df.loc[['bbox/AP', 'bbox/AP50', 'bbox/AP75', 'bbox/APl', 'bbox/APm', 'bbox/APs']]

,MAE,MoCo,DINO
bbox/AP,51.568317,48.808589,47.729175
bbox/AP50,72.238651,70.297221,68.117114
bbox/AP75,56.585393,53.562847,52.232559
bbox/APl,66.360938,64.827569,62.493270
bbox/APm,55.043170,52.412641,50.758096
bbox/APs,35.274414,32.792464,32.172995


In [6]:
df.loc[['segm/AP', 'segm/AP50', 'segm/AP75', 'segm/APl', 'segm/APm', 'segm/APs']]

,MAE,MoCo,DINO
segm/AP,45.842832,43.177882,42.022625
segm/AP50,69.703137,67.028010,65.558648
segm/AP75,49.945980,46.579699,45.273586
segm/APl,63.842282,62.860134,60.029667
segm/APm,48.769120,46.160463,44.639763
segm/APs,27.268124,23.782864,24.282897


In [7]:
results_pt = {'MoCo': [], 'DINO': [], 'MAE': []}

for key in results_pt:
    folder_name = "/checkpoint/sshkhr/experiments/detectron/" + key.lower() + "-pt/output/"
    log_file = folder_name + "metrics.json"

    if os.path.isfile(log_file): 
        with open(log_file, "r") as log:
            last_line = log.readlines()[-1]
            log_dict = literal_eval(last_line)
               
    results_pt[key] = log_dict

In [8]:
df_pt = pd.DataFrame.from_dict(results_pt['MAE'], orient='index', columns=['MAE'])
df_pt['MoCo'] = pd.Series(results_pt['MoCo'])
df_pt['DINO'] = pd.Series(results_pt['DINO'])

In [9]:
df_pt.loc[['bbox/AP', 'bbox/AP50', 'bbox/AP75', 'bbox/APl', 'bbox/APm', 'bbox/APs']]

,MAE,MoCo,DINO
bbox/AP,30.246264,28.746439,32.568707
bbox/AP50,49.856258,48.812903,53.708847
bbox/AP75,31.914899,29.858066,34.180488
bbox/APl,39.931006,40.210933,44.142811
bbox/APm,32.017861,29.987289,34.586185
bbox/APs,18.686022,15.497310,19.882592


In [10]:
df_pt.loc[['segm/AP', 'segm/AP50', 'segm/AP75', 'segm/APl', 'segm/APm', 'segm/APs']]

,MAE,MoCo,DINO
segm/AP,28.557045,26.666945,30.043295
segm/AP50,47.367876,45.613418,50.631923
segm/AP75,29.813092,27.368388,30.912970
segm/APl,41.726654,41.916242,45.774402
segm/APm,30.083755,27.143376,31.627910
segm/APs,14.365277,10.890953,14.373841


### Results with intermediate layers

In [11]:
resultsint = {'MAE': {}, 'MoCo': {}, 'DINO': {}}


for key in resultsint:

    folder_name = "/checkpoint/sshkhr/experiments/detectron/" + key.lower() + '-pt/output/'
    log_file = folder_name + "metrics.json"
        
    log_list = []
    print(log_file)
    if os.path.isfile(log_file):
        with open(log_file, "r") as log:
            for line in log:
                log_dict = literal_eval(line)
                if "bbox/AP" in log_dict.keys():
                    log_list.append(log_dict)
    
    df = pd.DataFrame(log_list)
    df = df.set_index('iteration')
    resultsint[key]['only'] = df
                    
    for i in range(9,12):
        folder_name = "/checkpoint/sshkhr/experiments/detectron/multi/" + key.lower() + '-pt/block'+str(i)+'-block11/output/'
        log_file = folder_name + "metrics.json"
            
        log_list = []
        print(log_file)
        if os.path.isfile(log_file):
            with open(log_file, "r") as log:
                for line in log:
                    log_dict = literal_eval(line)
                    if "bbox/AP" in log_dict.keys():
                        log_list.append(log_dict)
                
        df = pd.DataFrame(log_list)
        df = df.set_index('iteration')
        resultsint[key][i] = df

/checkpoint/sshkhr/experiments/detectron/mae-pt/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/mae-pt/block9-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/mae-pt/block10-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/mae-pt/block11-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/moco-pt/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/moco-pt/block9-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/moco-pt/block10-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/moco-pt/block11-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/dino-pt/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/dino-pt/block9-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/dino-pt/block10-block11/output/metrics.json
/checkpoint/sshkhr/experiments/detectron/multi/dino-pt/block11-block11/o

In [12]:
def dynamics_plot_individual(fig_dir, df1, name1, col):

    if not os.path.exists(fig_dir):
        os.makedirs(fig_dir)

    fig, ax = plt.subplots(figsize=(12, 9), dpi=300, sharex=True)

    colors = sns.color_palette("viridis", n_colors=3)
    colors.reverse()

    
    for i in range(9, 12):
        df1[i] = df1[i][~df1[i].index.duplicated(keep='last')]
        sns.lineplot(x='iteration', y=col, data = df1[i], color=colors[i-9], label='block.11 + block.'+str(i))

    sns.lineplot(x='iteration', y=col, data = df1['only'], color='r', label='block.11 only')

    '''
    axins = inset_axes(ax,  "25%", "40%" ,loc="upper right", borderpad=2.5)
    sns.lineplot(x='epoch', y=col, data = df1, ax = axins)
    sns.lineplot(x='epoch', y=col, data = df2, ax = axins)
    sns.lineplot(x='epoch', y=col, data = df3, ax = axins)
    axins.set_xlim([0,8])
    axins.set_xlabel("")
    axins.set_ylabel("")
    '''

    ax.set_xlabel("Mask-RCNN Training Epochs (over frozen ViT backbone)")
    ax.set_ylabel("MS-COCO "+col)
    ax.set_title(name1+' intermediate features marginal utility')
    
    plt.tight_layout()
    plt.show()

    fig.savefig(fig_dir+name1+'-top1.png')

    return fig

In [13]:
import itertools

In [ ]:
fig_dir = '../figures/coco-intermediateprobe/detection/'

prod = itertools.combinations(results.keys(), 3)

for name1 in resultsint.keys():
    print(name1)
    for col in ['bbox/AP', 'bbox/AP50', 'bbox/AP75', 'bbox/APl', 'bbox/APm', 'bbox/APs']:
        full_fig_dir = fig_dir + col + '/'
        fig = dynamics_plot_individual(full_fig_dir, resultsint[name1], name1, col=col)

In [ ]:
fig_dir = '../figures/coco-intermediateprobe/segmentation/'

prod = itertools.combinations(results.keys(), 3)

for name1 in resultsint.keys():
    print(name1)
    for col in ['segm/AP', 'segm/AP50', 'segm/AP75', 'segm/APl', 'segm/APm', 'segm/APs']:
        full_fig_dir = fig_dir + col + '/'
        fig = dynamics_plot_individual(full_fig_dir, resultsint[name1], name1, col=col)

In [16]:
prod = itertools.combinations(results.keys(), 3)

for name1 in resultsint.keys():
    print(name1)
    for col in ['bbox/AP']:#, 'bbox/AP50', 'bbox/AP75', 'bbox/APl', 'bbox/APm', 'bbox/APs']:
        print(col)
        
        for i in range(9, 12):
            df = resultsint[name1][i][~resultsint[name1][i].index.duplicated(keep='last')]
            print(i, col, df[col].max())

        df = resultsint[name1]['only'][~resultsint[name1]['only'].index.duplicated(keep='last')]
        print('only', col, df[col].max() )
        

MAE
bbox/AP
9 bbox/AP 31.46006785889347
10 bbox/AP 31.714336591383425
11 bbox/AP 30.469099840906537
only bbox/AP 30.27726416709634
MoCo
bbox/AP
9 bbox/AP 29.960781288958472
10 bbox/AP 20.324329375969487
11 bbox/AP 28.921014289504697
only bbox/AP 28.75073893277611
DINO
bbox/AP
9 bbox/AP 33.47588381020667
10 bbox/AP 32.92049757301459
11 bbox/AP 32.8338925638464
only bbox/AP 32.568706638529974


In [18]:
l = [31.71 - 30.27, 29.96-28.75, 33.47 - 32.56]
print(l)

[1.4400000000000013, 1.2100000000000009, 0.9099999999999966]
